In [1]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [2]:
path = "C:\\Users\\Anshul\\Documents\\Pluralsight\\Spark\\NYPD_7_Major_Felony_Incidents.csv"
data = sc.textFile(path)

In [3]:
data.take(10)

[u'OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1',
 u'1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"',
 u'2,c6245d4d,12/14/1968 12:20:00 AM,Saturday,Dec,14,1968,0,12,14,2008,GRAND LARCENY,FELONY,G,28,MANHATTAN,N.Y. POLICE DEPT,996470,232106,"(40.8037530600001, -73.955861904)"',
 u'3,716dbc6f,10/30/1970 03:30:00 PM,Friday,Oct,30,1970,15,10,31,2008,BURGLARY,FELONY,H,84,BROOKLYN,N.Y. POLICE DEPT,986508,190249,"(40.688874254, -73.9918594329999)"',
 u'4,638cd7b7,07/18/1972 11:00:00 PM,Tuesday,Jul,18,1972,23,7,19,2012,GRAND LARCENY OF MOTOR VEHICLE,FELONY,F,73,BROOKLYN,N.Y. POLICE DEPT,1005876,182440,"(40.6674141890001, -73.9220463899999)"',
 u'5,6e410287,05/21/1987 12

In [5]:
data.count()

1123466

In [6]:
header = data.first()
print(header)

OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1


In [7]:
dataWoHeader = data.filter(lambda x : x <> header)

In [9]:
print(dataWoHeader.first())

1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"


In [10]:
import csv
from StringIO import StringIO
from collections import namedtuple

fields = header.replace(' ','_').replace('/','_').split(",")

In [11]:
Crime = namedtuple('Crime', fields, verbose = True)

class Crime(tuple):
    'Crime(OBJECTID, Identifier, Occurrence_Date, Day_of_Week, Occurrence_Month, Occurrence_Day, Occurrence_Year, Occurrence_Hour, CompStat_Month, CompStat_Day, CompStat_Year, Offense, Offense_Classification, Sector, Precinct, Borough, Jurisdiction, XCoordinate, YCoordinate, Location_1)'

    __slots__ = ()

    _fields = ('OBJECTID', 'Identifier', 'Occurrence_Date', 'Day_of_Week', 'Occurrence_Month', 'Occurrence_Day', 'Occurrence_Year', 'Occurrence_Hour', 'CompStat_Month', 'CompStat_Day', 'CompStat_Year', 'Offense', 'Offense_Classification', 'Sector', 'Precinct', 'Borough', 'Jurisdiction', 'XCoordinate', 'YCoordinate', 'Location_1')

    def __new__(_cls, OBJECTID, Identifier, Occurrence_Date, Day_of_Week, Occurrence_Month, Occurrence_Day, Occurrence_Year, Occurrence_Hour, CompStat_Month, CompStat_Day, CompStat_Year, Offense, Offense_Classification, Sector, Precinct, Borough, Jurisdiction, XCoordinate, YCoordinate, Location_1):
        'Create new instance of Crime

In [12]:
def parse(row):
    reader = csv.reader(StringIO(row))
    row = reader.next()
    return Crime(*row)

In [13]:
crimes = dataWoHeader.map(parse)

In [14]:
crimes.first()

Crime(OBJECTID='1', Identifier='f070032d', Occurrence_Date='09/06/1940 07:30:00 PM', Day_of_Week='Friday', Occurrence_Month='Sep', Occurrence_Day='6', Occurrence_Year='1940', Occurrence_Hour='19', CompStat_Month='9', CompStat_Day='7', CompStat_Year='2010', Offense='BURGLARY', Offense_Classification='FELONY', Sector='D', Precinct='66', Borough='BROOKLYN', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='987478', YCoordinate='166141', Location_1='(40.6227027620001, -73.9883732929999)')

In [15]:
crimes.first().Offense

'BURGLARY'

In [16]:
crimes.map(lambda x : x.Offense).countByValue()

defaultdict(int,
            {'BURGLARY': 191369,
             'FELONY ASSAULT': 184042,
             'GRAND LARCENY': 428993,
             'GRAND LARCENY OF MOTOR VEHICLE': 101963,
             'MURDER & NON-NEGL. MANSLAUGHTE': 4574,
             'NA': 1,
             'RAPE': 13779,
             'ROBBERY': 198744})

In [18]:
crimes.map(lambda x : x.Occurrence_Year).countByValue()

defaultdict(int,
            {'': 244,
             '1905': 2,
             '1908': 1,
             '1910': 3,
             '1911': 1,
             '1912': 1,
             '1913': 4,
             '1914': 2,
             '1915': 3,
             '1920': 1,
             '1940': 1,
             '1945': 2,
             '1946': 1,
             '1950': 1,
             '1954': 1,
             '1955': 1,
             '1956': 1,
             '1958': 1,
             '1959': 1,
             '1960': 1,
             '1964': 1,
             '1965': 2,
             '1966': 7,
             '1968': 1,
             '1969': 1,
             '1970': 2,
             '1971': 1,
             '1972': 2,
             '1973': 5,
             '1974': 3,
             '1975': 2,
             '1976': 2,
             '1977': 3,
             '1978': 2,
             '1979': 6,
             '1980': 5,
             '1981': 1,
             '1982': 5,
             '1983': 1,
             '1984': 4,
             '1985': 8,
 

In [20]:
crimesFiltered = crimes.filter(lambda x : not (x.Offense == 'NA' or x.Occurrence_Year == ''))\
.filter(lambda x : int(x.Occurrence_Year) >= 2006)

In [22]:
crimesFiltered.map(lambda x : x.Occurrence_Year).countByValue()

defaultdict(int,
            {'2006': 127887,
             '2007': 120554,
             '2008': 117375,
             '2009': 106018,
             '2010': 105643,
             '2011': 107206,
             '2012': 111798,
             '2013': 111286,
             '2014': 106849,
             '2015': 102657})

In [23]:
crimes.first().Location_1 

'(40.6227027620001, -73.9883732929999)'

In [25]:
def extractCoords(location):
    location_lat = float(location[1:location.index(",")])
    location_lon = float(location[location.index(",") + 1: -1])
    return (location_lat, location_lon)

In [26]:
crimesFiltered.map(lambda x : extractCoords(x.Location_1))\
.reduce(lambda x,y : (min(x[0], y[0]), min(x[1],y[1])))

(40.112709974, -77.519206334)

In [27]:
crimesFiltered.map(lambda x : extractCoords(x.Location_1))\
.reduce(lambda x,y : (max(x[0], y[0]), max(x[1],y[1])))

(59.5805088160001, -73.700716685)

In [29]:
crimesFinal = crimesFiltered.filter(lambda x : extractCoords(x.Location_1)[0] >= 40.477399 \
                                    and extractCoords(x.Location_1)[0] <= 40.917577 \
                                    and extractCoords(x.Location_1)[1] >= -74.25909 \
                                    and extractCoords(x.Location_1)[1] <= -73.700009)

In [32]:
crimesFinal.map(lambda x : extractCoords(x.Location_1))\
.reduce(lambda x,y : (min(x[0], y[0]), min(x[1],y[1])))

(40.4987675300001, -74.255075543)

In [33]:
crimesFinal.map(lambda x : extractCoords(x.Location_1))\
.reduce(lambda x,y : (max(x[0], y[0]), max(x[1],y[1])))

(40.912723396, -73.700716685)

In [34]:
# Trend By Year
crimesFinal.map(lambda x : x.Occurrence_Year).countByValue()

defaultdict(int,
            {'2006': 127887,
             '2007': 120491,
             '2008': 117375,
             '2009': 106018,
             '2010': 105639,
             '2011': 107203,
             '2012': 111798,
             '2013': 111286,
             '2014': 106849,
             '2015': 102657})

In [36]:
crimesFinal.filter(lambda x : x.Offense == "BURGLARY").map(lambda x : x.Occurrence_Year).countByValue()

defaultdict(int,
            {'2006': 23069,
             '2007': 21715,
             '2008': 20732,
             '2009': 19441,
             '2010': 18700,
             '2011': 18860,
             '2012': 19309,
             '2013': 17419,
             '2014': 16832,
             '2015': 14967})

In [38]:
import gmplot
gmap = gmplot.GoogleMapPlotter(37.428, -122.145, 16).from_geocode("New York City")

In [41]:
b_lats = crimesFinal.filter(lambda x : x.Offense == "BURGLARY" and x.Occurrence_Year == '2015')\
.map(lambda x : extractCoords(x.Location_1)[0])\
.collect()

In [42]:
b_lons = crimesFinal.filter(lambda x : x.Offense == "BURGLARY" and x.Occurrence_Year == '2015')\
.map(lambda x : extractCoords(x.Location_1)[1])\
.collect()

In [44]:
gmap.scatter(b_lats, b_lons, '#DE1515', size = 40, marker = False)
gmap.draw("nymap.html")